# Modeling
ML Tasks

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Input

In [2]:
from sklearn.datasets import load_files

corpus = load_files("../data/")

doc_count = len(corpus.data)
print("Doc count:", doc_count)
assert doc_count is 60, "Wrong number of documents loaded, should be 60 (4 novels + 56 stories)"

Doc count: 60


## Vectorizer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tokenizer import TextWrangler

tfidf_stem = TfidfVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="stem"))
tfidf_stem.fit(corpus.data)
X_tfidf_stem = tfidf_stem.transform(corpus.data)

[nltk_data] Downloading package punkt to ../nltk/...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to ../nltk/...
[nltk_data]   Package wordnet is already up-to-date!


## Models

In [4]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF

n_topics = 15

lda = LatentDirichletAllocation(n_components=n_topics)
lsa = TruncatedSVD()
nmf = NMF()

In [5]:
lda_params = {"lda__learning_decay": [0.7, 0.9]}


## Pipeline

In [6]:
from sklearn.pipeline import Pipeline

lda_pipe = Pipeline([
    ("tfidf", tfidf_stem),
    ("lda", lda)
])

## Gridsearch

In [7]:
from sklearn.model_selection import GridSearchCV

lda_model = GridSearchCV(lda_pipe, param_grid=lda_params, cv=5, n_jobs=-1)

## Training

In [8]:
lda_model.fit(corpus.data)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...        random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lda__learning_decay': [0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
print(lda_model.best_params_)
lda_model.best_score_

{'lda__learning_decay': 0.9}


-5538.667962969713

## Evaluation

In [10]:
#print("Log Likelihood:", model.score(corpus.data))

In [12]:
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=tfidf_stem, lda_model=lda_model.best_estimator_.named_steps.lda, n_words=15)        

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,godfrey,bennet,ferguson,staunton,coronet,emswor,armstrong,presbury,overton,vampir,ralph,dod,dol,burnwel,cambridg
Topic 1,oldacr,baskervil,mcfarlane,moor,morst,norwood,stapleton,sholto,adair,barrym,jon,jona,ronald,thadde,mortim
Topic 2,majesty,briony,adl,norton,serpentin,rocket,godfrey,landau,guardsm,monic,warsaw,ormstein,kram,edgew,waylaid
Topic 3,amberley,josiah,ernest,lewisham,purlington,chess,mackinnon,futil,colourm,crockford,haymarket,disus,indel,someth,ingratitud
Topic 4,milverton,bayn,stackhurst,mcpherson,rond,leonardo,cubit,garc,hilton,murdoch,lion,merrilow,cag,hydra,elsy
Topic 5,bork,mccarthy,tregen,gibson,von,hosm,windibank,dunb,angel,mortim,boscomb,sternd,typewrit,shlessinger,lausan
Topic 6,hopkin,cunningham,carey,stanley,alec,acton,harpoon,smi,hut,willoughby,coram,nelig,yoxley,cabin,chatham
Topic 7,carruth,mela,woodley,openshaw,mycroft,charlington,greek,horsham,williamson,interpret,munro,smi,farnham,norbury,latim
Topic 8,said,man,com,hand,room,know,look,littl,tim,fac,did,think,hous,door,way
Topic 9,blessington,clair,nevil,lasc,brook,whitney,swandam,boon,bradstreet,is,kat,catalepsy,hugh,ced,wharf


In [13]:
import pyLDAvis
from pyLDAvis.sklearn import prepare
pyLDAvis.enable_notebook()

prepare(lda_model.best_estimator_.named_steps.lda, X_tfidf_stem, tfidf_stem, mds="tsne")

/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8      305.610474 -504.628082       1        1  88.685207
12    -435.545288  320.406342       2        1   1.291247
4      459.505798  176.816376       3        1   1.201036
5     -617.403809  -44.120617       4        1   1.192184
10     -89.845840  210.297836       5        1   0.984466
11    -275.684509 -585.361633       6        1   0.963297
0      -79.093231 -129.734253       7        1   0.877975
7      -11.502869 -429.974213       8        1   0.809722
1     -129.533401  518.290405       9        1   0.745613
6      204.992203 -196.927002      10        1   0.719188
13     512.012878 -167.038651      11        1   0.571978
14    -314.584106   27.503027      12        1   0.567624
9     -372.652649 -284.386841      13        1   0.511580
3      206.769516  416.899323      14        1   0.462551
2      163.880875   85.877380      15        1   0.416333, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
8807  Default  1.000000          turn  1.000000  30.0000  30.0000
9582  Default  1.000000           wom  1.000000  29.0000  29.0000
8424  Default  1.000000           tel  1.000000  28.0000  28.0000
7405  Default  1.000000          shal  1.000000  27.0000  27.0000
4635  Default  1.000000          just  1.000000  26.0000  26.0000
4972  Default  1.000000          long  1.000000  25.0000  25.0000
9661  Default  1.000000            ye  1.000000  24.0000  24.0000
5856  Default  1.000000            op  1.000000  23.0000  23.0000
3394  Default  1.000000        friend  1.000000  22.0000  22.0000
9543  Default  1.000000        window  1.000000  21.0000  21.0000
5096  Default  1.000000           mak  1.000000  20.0000  20.0000
5838  Default  1.000000           old  1.000000  19.0000  19.0000
2996  Default  1.000000            ey  1.000000  18.0000  18.0000
4730  Default  1.000000          lady  1.000000  17.0000  17.0000
6032  Default  1.000000           pap  1.000000  16.0000  16.0000
6077  Default  1.000000          pass  1.000000  15.0000  15.0000
4840  Default  1.000000          left  1.000000  14.0000  14.0000
5693  Default  1.000000         night  1.000000  13.0000  13.0000
8510  Default  1.000000       thought  1.000000  12.0000  12.0000
1547  Default  1.000000         clear  1.000000  11.0000  11.0000
5185  Default  1.000000           mat  1.000000  10.0000  10.0000
4905  Default  1.000000           lik  1.000000   9.0000   9.0000
420   Default  1.000000           ask  1.000000   8.0000   8.0000
7194  Default  1.000000           saw  1.000000   7.0000   7.0000
3924  Default  1.000000         heard  1.000000   6.0000   6.0000
3625  Default  1.000000          good  1.000000   5.0000   5.0000
2105  Default  1.000000           day  1.000000   4.0000   4.0000
5485  Default  1.000000          morn  1.000000   3.0000   3.0000
9144  Default  1.000000            us  1.000000   2.0000   2.0000
4867  Default  1.000000       lestrad  1.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
5894  Topic15  0.000821          ostl  0.060701   1.1784  -8.7712
3903  Topic15  0.000544     haymarket  0.056558   0.8373  -9.1830
2455  Topic15  0.000544         disus  0.056558   0.8373  -9.1830
1648  Topic15  0.000544       colourm  0.056558   0.8373  -9.1830
1957  Topic15  0.000544     crockford  0.056558   0.8373  -9.1830
7329  Topic15  0.000544          sedd  0.056635   0.8359  -9.1830
3757  Topic15  0.000544       gudgeon  0.056635   0.8359  -9.1830
5576  Topic15  0.000544        mutton  0.056645   0.8357  -9.1830
5058  Topic15  0.000544     mackinnon  0.056665   0.8354  -9.1830
1342  Topic15  0.000544           ced  0.056691   0.8349  -9.1830
2232  Topic15  0.000544    desborough  0.056710   0.8346  -9.1830
2227  Topic15  0.000544     derbyshir  0.056710   0.8346  -9.1830
9